<a href="https://colab.research.google.com/github/literolinguista/pesquisa-audio/blob/main/PesquisaAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install -q -U google-generativeai
!pip install -q SpeechRecognition pydub

import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import io

In [35]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('secretName')
genai.configure(api_key=GOOGLE_API_KEY)

In [36]:
model = genai.GenerativeModel(model_name = "gemini-1.0-pro")

# Prompt usado no Gemini para criar o código de upload e transcrição do arquivo de áudio:

> Escreva um código simples para Jupyter Notebook que receba o upload de um arquivo no formato exclusivo de MP3 e de no máximo 30 segundos, transcreva o áudio carregado e exiba o texto transcrito no formato de legenda com a indicação de tempo para cada frase.



In [39]:
# Função para transcrever o áudio
def transcrever_audio(audio_file):
  r = sr.Recognizer()
  with sr.AudioFile(audio_file) as source:
    audio = r.record(source)
  try:
    texto = r.recognize_google(audio, language='pt-BR')
    return texto
  except sr.UnknownValueError:
    return "Não foi possível entender o áudio"
  except sr.RequestError as e:
    return "Erro ao solicitar resultados do serviço de reconhecimento de fala Google; {0}".format(e)

# Função para gerar legendas com timestamp
def gerar_legendas(audio_file):
  sound_file = AudioSegment.from_file(audio_file, format="mp3")

  # Divide o áudio em chunks de silêncio
  chunks = split_on_silence(
      sound_file,
      min_silence_len=500,
      silence_thresh=sound_file.dBFS-16,
      keep_silence=100
  )

  legendas = []
  tempo_inicial = 0

  # Transcreve cada chunk e formata as legendas
  for i, chunk in enumerate(chunks):
    # Salva o chunk em um arquivo temporário
    chunk.export(f"chunk{i}.wav", format="wav")
    texto = transcrever_audio(f"chunk{i}.wav")

    # Calcula o tempo final do chunk
    tempo_final = tempo_inicial + len(chunk) / 1000
    legendas.append(f"{tempo_inicial:.2f} --> {tempo_final:.2f}\n{texto.capitalize()}\n")

    # Atualiza o tempo inicial para o próximo chunk
    tempo_inicial = tempo_final

  return legendas

# Interface para upload do arquivo MP3
uploaded = files.upload()

# Verifica se um arquivo foi carregado
if uploaded:
  for fn in uploaded.keys():
    # Verifica o formato do arquivo
    if not fn.lower().endswith('.mp3'):
      print("Por favor, faça o upload de um arquivo MP3.")
    else:
      # Verifica a duração do arquivo
      audio = AudioSegment.from_file(io.BytesIO(uploaded[fn]), format="mp3")
      if audio.duration_seconds > 30:
        print("O arquivo deve ter no máximo 30 segundos.")
      else:
        # Gera e exibe as legendas
        legendas = gerar_legendas(io.BytesIO(uploaded[fn]))
        for linha in legendas:
          print(linha)
else:
  print("Nenhum arquivo foi carregado.")

Saving recorteAlura30s.mp3 to recorteAlura30s (1).mp3
0.00 --> 6.79
E aí pessoal tudo bem sejam todos bem-vindos e bem-vindas ao 4 aqui da inversão da luna com o

6.79 --> 12.30
Eu sou a professora ana raquel eu sou aqui da fiap eu sou o professor e também dentista

12.30 --> 16.76
Eu tô aqui hoje com o fabrício que é da lora e o luciano martins que é da

16.76 --> 28.93
E hoje a gente vai ter uma aula bem prática bem revisão para a gente poder aprender como desenvolver né trabalhar aqui com o gmail que é a que a llm da google para a gente poder fazer passo a passo né pessoal



Pesquisa simples para exibição da palavra-chave no arquivo de texto transcrito.

In [52]:
palavra_chave = input("Digite a palavra que você procura: ")

encontrado = False
for i, linha in enumerate(legendas):
  if palavra_chave in linha:
    print(f"Palavra encontrada na linha {i+1}: {linha}")
    encontrado = True

if not encontrado:
  print(f"A palavra '{palavra_chave}' não foi encontrada no texto.")

Digite a palavra que você procura: sonho
A palavra 'sonho' não foi encontrada no texto.
